In [2]:
import numpy as np
import pandas as pd

 #  **Loading DATABASE (Mapped Positions)**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_excel('/content/drive/MyDrive/ArticleRecognition/Dataset/ArticleExtraction.xlsx', sheet_name='Mapped Positions')

In [5]:
data.sample(5)

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
1469,347416.0,2. 3. 7,Bekleidung Wandflächen - Fassadentafeln - FS2,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25948,9993,11000000,KVHLATTEN,7817.70,...,1.1,NaN,NaN,Liefern und montieren von Fassadenbekleidung m...,bekleidung wandflächen fassadentafeln fs2,liefern montieren fassadenbekleidung faserzeme...,bekleidung wandflächen fassadentafeln fs2;lief...,kvhlatten,kvh fi / ta nsi c24 din 1052 latten 4x6 6x6 6x...,kvhlatten;kvh fi / ta nsi c24 din 1052 latten ...
2132,347259.0,2. 3. 5,Bekleidung Wandfläche - Fassadentafel,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25920,9992,11000600,KVH6,18330.95,...,1.2,NaN,NaN,Liefern und montieren von Fassadenbekleidung m...,bekleidung wandfläche fassadentafel,liefern montieren fassadenbekleidung faserzeme...,bekleidung wandfläche fassadentafel;liefern mo...,kvh6,kvh fi / ta nsi c24 din 1052 breite 6 cm,kvh6;kvh fi / ta nsi c24 din 1052 breite 6 cm
1786,NaN,NaN,NaN,NaN,NaN,1976,1103,K324403003050,TT-Stück PN16,334.00,...,NaN,NaN,NaN,NaN,NaN,NaN,;,ttstück pn16,ttstück pn16 emailliert dn 80,ttstück pn16;ttstück pn16 emailliert dn 80
2622,344204.0,1.5.4.0030,Strasseneinlauf als Längsrekord 300/500 mit Sc...,NaN,NaN,24861,9969,2181110,STRA Schaft 5 b 2,63.00,...,2.2,NaN,NaN,NaN,strasseneinlauf längsrekord 300 / 500 schlammfang,NaN,strasseneinlauf längsrekord 300 / 500 schlammf...,stra schaft 5 b 2,stra schaft 5 b 295 mm,stra schaft 5 b 2;stra schaft 5 b 295 mm
1303,344751.0,1.4.4.0010,STZG-Kanal DN 100 ohne Erdarbeiten,NaN,NaN,24974,9971,30010,Steinzeug DN 100,1560.00,...,4.1,NaN,NaN,NaN,stzgkanal dn 100 erdarbeiten,NaN,stzgkanal dn 100 erdarbeiten;,steinzeug dn 100,steinzeug dn 100,steinzeug dn 100;steinzeug dn 100


In [6]:
df = data.copy()

In [7]:
df

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
0,35340.0,53.38.05.033110,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1742,1103,K141011032000,fugenband,14.3,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Ecke e...,zulage schweißung flache ecke pvcp sfd 24,zulage schweißung flachen ecke innenliegenden ...,zulage schweißung flache ecke pvcp sfd 24;zula...,fugenband,schweißung flache ecke pvcp sfd 24,fugenband;schweißung flache ecke pvcp sfd 24
1,35341.0,53.38.05.033120,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1743,1103,K141011032100,fugenband,22.9,...,1.1,NaN,NaN,Zulage für die Schweißung einer senkrechten Ec...,zulage schweißung senkrechte ecke pvcp sfd 24,zulage schweißung senkrechten ecke innenliegen...,zulage schweißung senkrechte ecke pvcp sfd 24;...,fugenband,schweißgsenkrechte ecke pvcp sfd 24,fugenband;schweißgsenkrechte ecke pvcp sfd 24
2,35342.0,53.38.05.033130,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1744,1103,K141011032200,fugenband,14.7,...,1.1,NaN,NaN,Zulage für die Schweißung eines flachen T-Stüc...,zulage schweißung flaches tstück pvcp sfd 24,zulage schweißung flachen tstücks innenliegend...,zulage schweißung flaches tstück pvcp sfd 24;z...,fugenband,schweißgflaches tstück pvcp sfd 24,fugenband;schweißgflaches tstück pvcp sfd 24
3,35343.0,53.38.05.033140,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1745,1103,K141011032300,fugenband,26.0,...,1.1,NaN,NaN,Zulage für die Schweißung eines senkrechten T-...,zulage schweißung senkrechtes tstück pvcp sfd 24,zulage schweißung senkrechten tstücks innenlie...,zulage schweißung senkrechtes tstück pvcp sfd ...,fugenband,schweißgsenkr tstück pvcp sfd 24,fugenband;schweißgsenkr tstück pvcp sfd 24
4,35344.0,53.38.05.033150,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1746,1103,K141011032400,fugenband,27.1,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Kreuzu...,zulage schweißung flache kreuzung pvcp sfd 24,zulage schweißung flachen kreuzung innenliegen...,zulage schweißung flache kreuzung pvcp sfd 24;...,fugenband,schweißgflache kreuzung pvcp sfd 24,fugenband;schweißgflache kreuzung pvcp sfd 24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3089,344554.0,1.1.1.0010,Baustelleneinrichtung,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",24972,9970,MK,NaN,100.0,...,4.3,NaN,NaN,"Einrichten, Unterhalten, Abbauen der Baustelle...",baustelleneinrichtung,einrichten unterhalten abbauen baustelleneinri...,baustelleneinrichtung;einrichten unterhalten a...,NaN,materialcontainer,;materialcontainer
3090,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,4.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenverbau ;großflächenv...,verbau graben,verbau graben 3 m tiefe,verbau graben ;verbau graben 3 m tiefe
3091,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,2.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenver

#**Extracting Useful columns**

In [8]:
df = df[['PositionShortText','LongText','ArticleNumber','Quantity']]

In [9]:
df.head()

,PositionShortText,LongText,ArticleNumber,Quantity
0,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Ecke e...,K141011032000,1.0
1,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer senkrechten Ec...,K141011032100,1.0
2,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Zulage für die Schweißung eines flachen T-Stüc...,K141011032200,1.0
3,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Zulage für die Schweißung eines senkrechten T-...,K141011032300,1.0
4,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Kreuzu...,K141011032400,1.0


In [10]:
df['LongText'][250]

nan

#**Text Preprocessing**

## 1.Handling Missing Data

In [11]:
# Identify missing data
missing_data = df.isnull().any(axis=1)

# Filter out rows with missing data
data_cleaned = df[~missing_data]

# Drop the index associated with the rows containing missing data
data_cleaned = data_cleaned.reset_index(drop=True)

In [12]:
data_cleaned.isnull().sum()

PositionShortText    0
LongText             0
ArticleNumber        0
Quantity             0
dtype: int64

##2.Lowercasing the Text

In [13]:
 # Convert all string columns to lowercase
data_cleaned = data_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)

## 3.Remove punctuation and special characters

In [14]:
import re

# Function to remove punctuation and special characters from text
def remove_punctuation_special_characters(text):
    # Define regex pattern to match any character that is not a word character or whitespace
    pattern = r'[^\w\säöüßÄÖÜ]'  # Including German special characters ä, ö, ü, ß, Ä, Ö, and Ü

    # Use regex to substitute all non-word characters and non-whitespace characters with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# Remove punctuation and special characters from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_punctuation_special_characters)

# Remove punctuation and special characters from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_punctuation_special_characters)



## 4.Removing Stopwords

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords list for German if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Get the set of German stopwords
stop_words_german = set(stopwords.words('german'))

# Function to remove stopwords from text
def remove_stopwords_german(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words_german]
    return ' '.join(filtered_tokens)

# Remove stopwords from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_stopwords_german)

# Remove stopwords from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_stopwords_german)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 5.Lemmetization

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [18]:
!pip install pattern


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332702 sha256=9a9b88f44d824ade5dfa2d91d32676d0398a22b7f5d74f8469a1ebe83d506cff
  Stored in dire

In [21]:
from pattern.de import lemma

# Function to lemmatize text in German
def lemmatize_text_german(text):
    tokens = text.split()  # Split text into tokens
    lemmatized_tokens = []
    for token in tokens:
        try:
            lemmatized_token = lemma(token)
            lemmatized_tokens.append(lemmatized_token)
        except StopIteration:
            # Handle StopIteration exception
            # You can choose to ignore the token or handle it differently
            pass
    lemmatized_text = ' '.join(lemmatized_tokens)  # Join lemmatized tokens back into text
    return lemmatized_text

# Lemmatize 'LongText' column in German
data_cleaned['LongText'] = data_cleaned['LongText'].apply(lemmatize_text_german)

# Lemmatize 'PositionShortText' column in German
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(lemmatize_text_german)



## 6.Tokenization

In [22]:
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources for German language if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Tokenize 'LongText' column in German
data_cleaned['LongText_tokens'] = data_cleaned['LongText'].apply(lambda x: word_tokenize(x, language='german'))

# Tokenize 'PositionShortText' column in German
data_cleaned['PositionShortText_tokens'] = data_cleaned['PositionShortText'].apply(lambda x: word_tokenize(x, language='german'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [38]:
data_cleaned


,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens,ArticleNumber_encoded
0,zuliegen schweißungen flachen ecken pvcpen sfd...,zuliegen schweißungen flachen ecken innenliege...,k141011032000,1.0,"[zuliegen, schweißungen, flachen, ecken, innen...","[zuliegen, schweißungen, flachen, ecken, pvcpe...",246
1,zuliegen schweißungen senkrechen ecken pvcpen ...,zuliegen schweißungen senkrechen ecken innenli...,k141011032100,1.0,"[zuliegen, schweißungen, senkrechen, ecken, in...","[zuliegen, schweißungen, senkrechen, ecken, pv...",247
2,zuliegen schweißungen flachesen tstücken pvcpe...,zuliegen schweißungen flachen tstücksen innenl...,k141011032200,1.0,"[zuliegen, schweißungen, flachen, tstücksen, i...","[zuliegen, schweißungen, flachesen, tstücken, ...",248
3,zuliegen schweißungen senkrechtesen tstücken p...,zuliegen schweißungen senkrechen tstücksen inn...,k141011032300,1.0,"[zuliegen, schweißungen, senkrechen, tstücksen...","[zuliegen, schweißungen, senkrechtesen, tstück...",249
4,zuliegen schweißungen flachen kreuzungen pvcpe...,zuliegen schweißungen flachen kreuzungen innen...,k141011032400,1.0,"[zuliegen, schweißungen, flachen, kreuzungen, ...","[zuliegen, schweißungen, flachen, kreuzungen, ...",250
...,...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen],303
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]",195
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]",195
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen],236


In [24]:
# Save the DataFrame to an Excel file
data_cleaned.to_excel('/content/drive/MyDrive/data_cleaned.xlsx', index=False)


In [25]:
data_cleaned

,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens
0,zuliegen schweißungen flachen ecken pvcpen sfd...,zuliegen schweißungen flachen ecken innenliege...,k141011032000,1.0,"[zuliegen, schweißungen, flachen, ecken, innen...","[zuliegen, schweißungen, flachen, ecken, pvcpe..."
1,zuliegen schweißungen senkrechen ecken pvcpen ...,zuliegen schweißungen senkrechen ecken innenli...,k141011032100,1.0,"[zuliegen, schweißungen, senkrechen, ecken, in...","[zuliegen, schweißungen, senkrechen, ecken, pv..."
2,zuliegen schweißungen flachesen tstücken pvcpe...,zuliegen schweißungen flachen tstücksen innenl...,k141011032200,1.0,"[zuliegen, schweißungen, flachen, tstücksen, i...","[zuliegen, schweißungen, flachesen, tstücken, ..."
3,zuliegen schweißungen senkrechtesen tstücken p...,zuliegen schweißungen senkrechen tstücksen inn...,k141011032300,1.0,"[zuliegen, schweißungen, senkrechen, tstücksen...","[zuliegen, schweißungen, senkrechtesen, tstück..."
4,zuliegen schweißungen flachen kreuzungen pvcpe...,zuliegen schweißungen flachen kreuzungen innen...,k141011032400,1.0,"[zuliegen, schweißungen, flachen, kreuzungen, ...","[zuliegen, schweißungen, flachen, kreuzungen, ..."
...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]


In [39]:
from keras.preprocessing.text import Tokenizer

# Initialize a tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the LongText_tokens and PositionShortText_tokens
all_tokens = data_cleaned['LongText_tokens'].tolist() + data_cleaned['PositionShortText_tokens'].tolist()
tokenizer.fit_on_texts(all_tokens)

# Convert text to sequences of numbers
data_cleaned['LongText_sequences'] = tokenizer.texts_to_sequences(data_cleaned['LongText_tokens'])
data_cleaned['PositionShortText_sequences'] = tokenizer.texts_to_sequences(data_cleaned['PositionShortText_tokens'])


In [40]:
data_cleaned

,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens,ArticleNumber_encoded,LongText_sequences,PositionShortText_sequences
0,zuliegen schweißungen flachen ecken pvcpen sfd...,zuliegen schweißungen flachen ecken innenliege...,k141011032000,1.0,"[zuliegen, schweißungen, flachen, ecken, innen...","[zuliegen, schweißungen, flachen, ecken, pvcpe...",246,"[431, 722, 1104, 1303, 1011, 1012, 723, 555, 1...","[431, 722, 1104, 1303, 723, 724, 438]"
1,zuliegen schweißungen senkrechen ecken pvcpen ...,zuliegen schweißungen senkrechen ecken innenli...,k141011032100,1.0,"[zuliegen, schweißungen, senkrechen, ecken, in...","[zuliegen, schweißungen, senkrechen, ecken, pv...",247,"[431, 722, 1041, 1303, 1011, 1012, 723, 555, 1...","[431, 722, 1041, 1303, 723, 724, 438]"
2,zuliegen schweißungen flachesen tstücken pvcpe...,zuliegen schweißungen flachen tstücksen innenl...,k141011032200,1.0,"[zuliegen, schweißungen, flachen, tstücksen, i...","[zuliegen, schweißungen, flachesen, tstücken, ...",248,"[431, 722, 1104, 1497, 1011, 1012, 723, 555, 1...","[431, 722, 2015, 1673, 723, 724, 438]"
3,zuliegen schweißungen senkrechtesen tstücken p...,zuliegen schweißungen senkrechen tstücksen inn...,k141011032300,1.0,"[zuliegen, schweißungen, senkrechen, tstücksen...","[zuliegen, schweißungen, senkrechtesen, tstück...",249,"[431, 722, 1041, 1497, 1011, 1012, 723, 555, 1...","[431, 722, 2016, 1673, 723, 724, 438]"
4,zuliegen schweißungen flachen kreuzungen pvcpe...,zuliegen schweißungen flachen kreuzungen innen...,k141011032400,1.0,"[zuliegen, schweißungen, flachen, kreuzungen, ...","[zuliegen, schweißungen, flachen, kreuzungen, ...",250,"[431, 722, 1104, 1105, 1011, 1012, 723, 555, 1...","[431, 722, 1104, 1105, 723, 724, 438]"
...,...,...,...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen],303,"[389, 624, 618, 272, 625, 519, 72, 516, 626, 6...",[272]
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]",195,"[1480, 1176, 38, 36, 1481, 1482, 1020, 1483, 1...","[614, 1176]"
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]",195,"[1480, 1176, 38, 36, 1481, 1482, 1020, 1483, 1...","[614, 1176]"
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen],236,"[389, 624, 618, 272, 625, 519, 72, 516, 626, 6...",[272]


#**Train-Test Split of the data**

In [41]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = data_cleaned[['LongText_sequences', 'PositionShortText_sequences' ]]  # Features
y = data_cleaned[['ArticleNumber','Quantity']]  # Target variable (replace 'Target_Column' with your actual target column name)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Train set shape: (1606, 2) (1606, 2)
Test set shape: (402, 2) (402, 2)


In [42]:
X_train.iloc[0:6]


,LongText_sequences,PositionShortText_sequences
1249,"[38, 75, 551, 205, 231, 144, 167, 168, 162, 16...","[554, 205, 1, 1215]"
261,"[124, 45, 125, 126, 127, 7, 8, 10, 7, 86, 17, ...","[120, 222, 6, 428]"
607,"[556, 619, 383, 587, 52]","[620, 1716, 989]"
973,"[124, 45, 125, 126, 127, 7, 8, 10, 7, 86, 17, ...","[120, 154, 50]"
240,"[38, 75, 399, 522, 205, 136, 7, 8, 221, 16, 66...","[381, 205, 1, 650]"
819,"[38, 75, 218, 266, 71, 8, 221, 207, 55, 196, 1...","[210, 225, 136, 400]"


In [43]:
y_train.head()

,ArticleNumber,Quantity
1249,24900100,0.21
261,40001400,5022.00
607,k462052003500,1.00
973,69910600,6820.00
240,24100000,3.00


##Storing each of it as separate variables

In [45]:
# Store X_train and X_test columns as separate variables
X_train_LongText_sequences = X_train['LongText_sequences']
X_train_PositionShortText_sequences = X_train['PositionShortText_sequences']

X_test_LongText_sequences = X_test['LongText_sequences']
X_test_PositionShortText_sequences = X_test['PositionShortText_sequences']

# Store 'Quantity' column for train and test sets
y_train_quantity = y_train['Quantity']
y_test_quantity = y_test['Quantity']

# Store 'ArticleNumber' column for train and test sets
y_train_article_number = y_train.copy()
y_test_article_number = y_test.copy()



#**Encoding The Article Number**


In [46]:
data_cleaned['ArticleNumber'] = data_cleaned['ArticleNumber'].astype(str)


In [47]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder on the article numbers and transform them
data_cleaned['ArticleNumber_encoded'] = label_encoder.fit_transform(data_cleaned['ArticleNumber'])

# Print the mapping of original article numbers to encoded values
print("Mapping of original article numbers to encoded values:")
for original_value, encoded_value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{original_value}: {encoded_value}")


Mapping of original article numbers to encoded values:
00-bee-0010: 0
00-bee-0020: 1
00-bee-0021: 2
00-bee-0022: 3
00-bee-0030: 4
00-bee-0040: 5
00-bee-0050: 6
00-bee-0055: 7
00-bee-0060: 8
00-bee-0065: 9
00-bee-0070: 10
00-bee-0080: 11
00-bee-0100: 12
00-bee-0120: 13
00-bee-0130: 14
00-mat-0010: 15
02-dep-0010: 16
02-fra-0010: 17
02-fra-0020: 18
02-sch-0010: 19
02-sch-0030: 20
02-sch-0040: 21
02-ver-0010: 22
09-pvc-0010: 23
09-pvc-0020: 24
09-pvc-0030: 25
09-pvc-0040: 26
09-pvc-0050: 27
09-pvc-0060: 28
09-pvc-0070: 29
09-pvc-0080: 30
09-pvc-0090: 31
09-pvc-0100: 32
09-pvc-0110: 33
09-pvc-0120: 34
09-pvc-0130: 35
09-pvc-0140: 36
09-pvc-0190: 37
09-pvc-0200: 38
09-pvc-0210: 39
09-pvc-0220: 40
09-pvc-0230: 41
09-sch-0010: 42
09-sch-0020: 43
09-sch-0030: 44
09-sch-0040: 45
09-sch-0050: 46
09-sch-0110: 47
09-sch-0120: 48
09-sch-0140: 49
09-sch-0170: 50
09-sch-0190: 51
09-sch-0200: 52
09-sch-0210: 53
09-sch-0220: 54
09-sch-0230: 55
09-sch-0250: 56
09-sch-0260: 57
09-sch-0270: 58
09-sch-0280

In [48]:
encoded_variable_counts = data_cleaned['ArticleNumber_encoded'].value_counts()
print(encoded_variable_counts)


72     123
183     96
182     96
73      96
203     54
      ... 
296      1
297      1
298      1
247      1
71       1
Name: ArticleNumber_encoded, Length: 306, dtype: int64


In [49]:
# Group the DataFrame by 'ArticleNumber_encoded' and aggregate the indexes for each group
indexes_by_article_number = data_cleaned.groupby('ArticleNumber_encoded').apply(lambda x: list(x.index))

# Print the indexes for each article number
for article_number, indexes in indexes_by_article_number.items():
    print(f"Article Number {article_number}: Indexes {indexes}")


Article Number 0: Indexes [704, 1110]
Article Number 1: Indexes [705, 1111]
Article Number 2: Indexes [706, 1112]
Article Number 3: Indexes [707, 1113]
Article Number 4: Indexes [653, 657]
Article Number 5: Indexes [654, 658]
Article Number 6: Indexes [1140, 1548]
Article Number 7: Indexes [1141, 1549]
Article Number 8: Indexes [1142, 1550]
Article Number 9: Indexes [1143, 1551]
Article Number 10: Indexes [1144, 1552]
Article Number 11: Indexes [1145, 1553]
Article Number 12: Indexes [655, 656, 659, 660]
Article Number 13: Indexes [1530, 1533]
Article Number 14: Indexes [1146, 1554]
Article Number 15: Indexes [59, 1147]
Article Number 16: Indexes [1531, 1532, 1534, 1535]
Article Number 17: Indexes [644, 645, 650, 651]
Article Number 18: Indexes [646, 647, 648, 649, 652, 1079, 1080, 1081]
Article Number 19: Indexes [60, 1148]
Article Number 20: Indexes [61, 62, 63, 64, 1149, 1150, 1578, 1579]
Article Number 21: Indexes [65, 1580]
Article Number 22: Indexes [1048, 1049, 1050, 1051, 1508,

In [34]:
data_cleaned['ArticleNumber'][42]

'k485512101200'

In [51]:
 # Extract long text sequences and position short text sequences from the DataFrame
long_text_sequences = data_cleaned['LongText_sequences'].tolist()
position_short_text_sequences = data_cleaned['PositionShortText_sequences'].tolist()


In [54]:
# Convert each integer sequence into a list of lists
tokenized_long_text_sequences = [[token] for token in long_text_sequences]
tokenized_position_short_text_sequences = [[token] for token in position_short_text_sequences]



In [57]:
from collections import Counter

# Assuming tokenized_long_text_sequences and tokenized_position_short_text_sequences are lists of tokenized sequences
# Remove the extra brackets around each sequence
tokenized_long_text = tokenized_long_text_sequences  # List of tokenized long text sequences
tokenized_short_text = tokenized_position_short_text_sequences  # List of tokenized short text sequences

# Concatenate tokenized sequences
all_tokenized_texts = tokenized_long_text + tokenized_short_text

# Flatten the list of tokenized sequences and handle nested lists
all_tokens = [token for sequence in all_tokenized_texts for token_list in sequence for token in token_list]

# Count the frequency of each token
token_counts = Counter(all_tokens)

# Calculate the vocabulary size
vocabulary_size = len(token_counts)

print("Vocabulary size:", vocabulary_size)


Vocabulary size: 2138


# Decoder

In [35]:
# Decode the encoded article numbers back to their original values
decoded_article_numbers = label_encoder.inverse_transform(data_cleaned['ArticleNumber_encoded'])

# Print the decoded article numbers
print("Decoded article numbers:")
print(decoded_article_numbers)


Decoded article numbers:
['k141011032000' 'k141011032100' 'k141011032200' ... '3303' 'bc' 'bc']


In [36]:
da = pd.read_excel('/content/drive/MyDrive/ArticleRecognition/Dataset/ArticleExtraction.xlsx', sheet_name='Article Master')

In [37]:
da


,ArticleNumber,ShortName,Description,BasePrice,Unit,DescriptionWithAbbreviation
0,00-BEE-0100,BEE,Kranmonteura1,50.000,Std,Kranmonteura1
1,00-BEE-0120,BE,Bauzaun-Elemente,15.123,Stck,Bauzaun-Elemente
2,00-TRE-0010,DIESEL A,Diesel A,5.000,OKm,Diesel A
3,02-DEP-0010,DEPO,Deponiegebühren Erdaushub je to,2.580,to,Deponiegebühren Erdaushub je to
4,02-FRA-0010,FRACHT,Fracht Schotter,3.500,to,Fracht Schotter
...,...,...,...,...,...,...
1214,sf156,SIKAFLOOR 156,Sikafloor 156,1448.335,kg,Sikafloor 156
1215,Stahlwinkel,NaN,"Schlosserteile, Winkel",0.000,Psch,"Schlosserteile, Winkel"
1216,Verbinder,NaN,Verbindungsmittel,0.000,Psch,Verbindungsmittel
1217,yxc123,NaN,Telefonkabel,0.000,m,Telefonkabel
